In [1]:
import sys

# Add path for python to look into for modules installed using pip

sys.path.append('/usr/local/lib/python3.7/site-packages/')

import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import datetime

quandl.ApiConfig.api_key = "n2tNssPxEFC9-Ad79fo-" # keep this private

figsize = (15, 8)

pd.set_option('display.max_columns', None)

In [2]:
# Start/end dates

start = '2010-10-15'
end = '2018-10-15'

oos_start = end
oos_end = '2019-10-15'

In [3]:
##### Rifinitiv #####

# The following values are populated for you by Data Science Accelerator. 
# They represent your demo-level access to the data.
# Please don't share this with anyone

RESOURCE_ENDPOINT = 'https://dsa-stg-edp-api.fr-nonprod.aws.thomsonreuters.com/data/environmental-social-governance/v1/views/scores-full'
access_token = 'uGR7cxvvqJ4mgWwva5pPN184iGGigBhY8g4ThFu0' # your personal key for Data Science Accelerator access to ESG

def get_data_request(url, requestData):
    '''HTTP GET request'''
    dResp = requests.get(url, headers = {'X-api-key': access_token}, params = requestData);       

    if dResp.status_code != 200:
        raise ValueError("Unable to get data. Code %s, Message: %s" % (dResp.status_code, dResp.text));
    else:
        print("Data access successful")
        jResp = json.loads(dResp.text);
        return jResp

def get_data(ric):
    '''Gets ESG scores for a specific RIC (company) code'''
    
    requestData = {
    "universe": ric
    };

    jResp = get_data_request(RESOURCE_ENDPOINT, requestData)

    data = jResp["data"]
    headers = jResp["headers"]    

    names = [headers[x]['title'] for x in range(len(headers))]

    df = pd.DataFrame(data, columns=names )
    
    return df

# Updating the API endpoint 

RESOURCE_ENDPOINT = 'https://dsa-stg-edp-api.fr-nonprod.aws.thomsonreuters.com/data/environmental-social-governance/v1/views/measures-full'

In [4]:
# Define essential loop function

def aggregate(asset1, asset2, start_date=start, end_date=end):
    
    # Quandl
    
    quandl_data = quandl.get('EOD/'+asset1, start_date=start_date, end_date=end_date)
    quandl_data['Date'] = pd.to_datetime(quandl_data.index, format='%Y-%m-%d')
    quandl_data['Year'] = quandl_data['Date'].dt.year
    quandl_yearly = quandl_data.groupby(['Year']).mean()
    
    # Refinitiv
    
    refinitiv_data = get_data(asset2)
    refinitiv_data['Date'] = pd.to_datetime(refinitiv_data['Period End Date'], format='%Y-%m-%d')
    refinitiv_data['Year'] = refinitiv_data['Date'].dt.year
    refinitiv_data['Period End Date'] = pd.to_datetime(refinitiv_data['Period End Date'], format='%Y-%m-%d')
    refinitiv_data = refinitiv_data[(refinitiv_data['Period End Date'] >= start) & 
                                    (refinitiv_data['Period End Date'] <= end)]
    refinitiv_data.set_index('Year', inplace=True)
    refinitiv_data.dropna(subset=['TRESG Combined Score'], inplace=True)
    
    all_data = quandl_yearly.join(refinitiv_data, on='Year', how='left')
    
    return all_data

In [5]:
assets_quandl =["AAPL","MSFT","AMZN","FB", "NFLX","ENDP","CSCO","INTC","CMCSA","SIRI","PLUG","ETFC"]

assets_ref = ["AAPL.O","MSFT.O","AMZN.O","FB.O","NFLX.O","ENDP.O","CSCO.O","INTC.O","CMCSA.O","SIRI.O","PLUG.O","ETFC.O"]

datasets = []

for asset1, asset2 in zip(assets_quandl,assets_ref):
    
    print(asset1, ',', asset2)
    
    data_tmp = aggregate(asset1, asset2)
    datasets.append(data_tmp)
    
    print(data_tmp.shape, '\n')
    
    if asset1 == assets_quandl[-1] and asset2 == assets_ref[-1]:
        print('Done')
    

AAPL , AAPL.O
Data access successful
(9, 344) 

MSFT , MSFT.O
Data access successful
(9, 358) 

AMZN , AMZN.O
Data access successful
(9, 309) 

FB , FB.O
Data access successful
(7, 312) 

NFLX , NFLX.O
Data access successful
(9, 287) 

ENDP , ENDP.O
Data access successful
(9, 291) 

CSCO , CSCO.O
Data access successful
(9, 359) 

INTC , INTC.O
Data access successful
(9, 359) 

CMCSA , CMCSA.O
Data access successful
(9, 316) 

SIRI , SIRI.O
Data access successful
(9, 289) 

PLUG , PLUG.O
Data access successful
(9, 267) 

ETFC , ETFC.O
Data access successful
(9, 281) 

Done


In [6]:
# Get all column names

col_names = []

for d in datasets:
    col_names.append(d.columns)
    
# Intersection of list of lists

unique_colnames = list(set.intersection(*map(set, col_names)))

In [7]:
# Subset dataframes in datasets

new_data = []

for d in datasets:
    df_tmp = d[np.intersect1d(d.columns, unique_colnames)]
    df_tmp = df_tmp.loc[:, ~df_tmp.columns.duplicated()]
    new_data.append(df_tmp)
    
# Merge dataframes in new_data list

all_data = pd.concat(new_data, axis=0)

In [8]:
all_data.shape

(106, 258)

In [9]:
all_data.head()

,Accounting Controversies,Adj_Close,Adj_High,Adj_Low,Adj_Open,Adj_Volume,Advance Notice Period Days,Advance Notice for Shareholder Proposals,Agrochemical 5 % Revenue,Agrochemical Products,Alcohol,Animal Testing,Animal Testing Cosmetics,Animal Testing Reduction,Announced Layoffs To Total Employees,Anti Takeover Devices Above Two,Anti-Competition Controversies Count,Anti-Personal Landmines,Anti-competition Controversies,Armaments,Audit Board Committee,Audit Committee Expertise,Audit Committee Independence,Audit Committee Mgt Independence,Audit Committee NonExecutive Members,Auditor Independence Rotation,Average Board Tenure,Biodiversity Impact Reduction,Board Attendance,Board Background and Skills,Board Functions Policy,"Board Gender Diversity, Percent",Board Individual Reelection,Board Meeting Attendance Average,Board Member Affiliations,Board Member Compensation,Board Member Membership Limits,Board Member Term Duration,Board Size,Board Size More Ten Less Eight,"Board Specific Skills, Percent",Board Structure Policy,Board Structure Type,"Bribery, Corruption and Fraud Controversies",CEO Board Member,CEO Compensation Link to TSR,CEO-Chairman Separation,CO2 Estimation Method,CSR Strategy Score,CSR Sustainability Committee,CSR Sustainability External Audit,CSR Sustainability Reporting,Chairman is ex-CEO,Classified Board Structure,Climate Change Commercial Risks Opportunities,Close,Cluster Bombs,Committee Meetings Attendance Average,Community Score,Compensation Board Committee,Compensation Committee Independence,Compensation Committee Mgt Independence,Compensation Committee NonExecutive Members,Compensation Improvement Tools,Consumer Complaints Controversies,Contraceptives,Corporate Governance Board Committee,Corporate Responsibility Awards,Crisis Management Systems,Date,Day Care Services,Director Election Majority Requirement,Diseases of the Developing World,Dividend,Dual Class Stock,ESG Controversies Score,ESG Period Last Update Date,Earnings Restatement,Eco-Design Products,Elimination of Cumulative Voting Rights,Embryonic Stem Cell Research,Emissions Score,Emissions Trading,Employee Engagement Voluntary Work,Employees Health & Safety OHSAS 18001,Employees Health & Safety Team,Env Supply Chain Partnership Termination,Environment Management Team,Environment Management Training,Environment Pillar Score,Environmental Assets Under Mgt,Environmental Controversies,Environmental Expenditures Investments,Environmental Investments Initiatives,Environmental Materials Sourcing,Environmental Partnerships,Environmental Products,Environmental Project Financing,Environmental Restoration Initiatives,Environmental Supply Chain Management,Equal Voting Rights,Equator Principles,Estimated CO2 Equivalents Emission Total,Ethical Trading Initiative ETI,Executive Compensation Controversies,Executive Compensation LT Objectives,Executive Compensation Policy,Executive Individual Compensation,"Executive Members Gender Diversity, Percent",External Consultants,Extractive Industries Transparency Initiative,Flexible Working Hours,Fundamental Human Rights ILO UN,GMO Products,GRI Report Guidelines,Gambling,Global Compact Signatory,Golden Parachute,Governance Pillar Score,Green Buildings,HIV-AIDS Program,Health & Safety Policy,Health & Safety Training,Healthy Food or Products,High,Highest Remuneration Package,Human Rights Breaches Contractor,Human Rights Contractor,Human Rights Policy,Human Rights Score,Hybrid Vehicles,ISO 14000 or EMS,ISO 9000,Improvement Tools Business Ethics,Independent Board Members,Innovation Score,Insider Dealings Controversies,Instrument,Integrated Strategy in MD&A,Internal Audit Department Reporting,Internal Promotion,Labeled Wood,Land Environmental Impact Reduction,Limitation of Director Liability,Limited Shareholder Rights to Call Meetings,Lobbying Contribution Amount,Low,Management Departures,Management Score,Management Training,Minimum Number of Shares to Vote,NOx and SOx Emissions Reduction,Net Employment Creation,Noise Reduction,Nomin

In [10]:
all_data.to_csv('aggr_data.csv', sep = ',')